## Kindle Review Sentiment Analyis

**About Dataset**

Context This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content 5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset. Columns

asin - ID of the product, like B000FA64PK
helpful - helpfulness rating of the review - example: 2/3.
overall - rating of the product.
reviewText - text of the review (heading).
reviewTime - time of the review (raw).
reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
reviewerName - name of the reviewer.
summary - summary of the review (description).
unixReviewTime - unix timestamp.
Acknowledgements This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration

Sentiment analysis on reviews.
Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
Fake reviews/ outliers.
Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
Any other interesting analysis

**Best Practises**

1.Preprocessing And Cleaning

2.Train Test Split

3.BOW,TFIDF

4.Train ML algorithms

In [1]:
# 1. Import Libraries
# ================================
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\palla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# 2. Load Dataset
# ================================
df = pd.read_csv("Data\\all_kindle_review.csv")

df = df[['reviewText','rating']]
df.dropna(inplace=True)

display(df.head())
print("Shape:", df.shape)

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


Shape: (12000, 2)


In [3]:
# 3. Create Sentiment Labels
# rating > 3 = Positive (1)
# rating <= 3 = Negative (0)
# ================================
df['sentiment'] = df['rating'].apply(lambda x: 1 if x > 3 else 0)
df[['rating','sentiment']].head()

,rating,sentiment
0,3,0
1,5,1
2,3,0
3,3,0
4,4,1


In [4]:
# 4. Text Cleaning Function
# (Fix stopwords: keep 'not', 'no', 'never')
# ================================
stop_words = set([w for w in stopwords.words("english")
                  if w not in ["not", "no", "never"]])

def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()        # remove HTML
    text = re.sub(r"[^a-zA-Z ]", "", text)                      # keep only letters
    text = text.lower()                                         # lower case
    words = [w for w in text.split() if w not in stop_words]    # remove stopwords
    return " ".join(words)

# Apply cleaning
df['cleaned_text'] = df['reviewText'].apply(clean_text)
display(df.head())

,reviewText,rating,sentiment,cleaned_text
0,"Jace Rankin may be short, but he's nothing to ...",3,0,jace rankin may short hes nothing mess man hau...
1,Great short read. I didn't want to put it dow...,5,1,great short read didnt want put read one sitti...
2,I'll start by saying this is the first of four...,3,0,ill start saying first four books wasnt expect...
3,Aggie is Angela Lansbury who carries pocketboo...,3,0,aggie angela lansbury carries pocketbooks inst...
4,I did not expect this type of book to be in li...,4,1,not expect type book library pleased find pric...


In [6]:
# 5. TF-IDF Vectorization
# ================================
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['cleaned_text'])
y = df['sentiment']

print("TF-IDF Shape:", X.shape)

TF-IDF Shape: (12000, 5000)


In [7]:
# 6. Train–Test Split
# ================================
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [8]:
# 7. Train Model (Logistic Regression)
# ================================
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

print("Model trained successfully!")

Model trained successfully!


In [9]:
# 8. Evaluate Model
# ================================
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.83
              precision    recall  f1-score   support

           0       0.84      0.81      0.83      1190
           1       0.82      0.85      0.83      1210

    accuracy                           0.83      2400
   macro avg       0.83      0.83      0.83      2400
weighted avg       0.83      0.83      0.83      2400



In [10]:
# 9. Save Model + Vectorizer
# ================================
import pickle

pickle.dump(tfidf, open("vectorizer.pkl", "wb"))
pickle.dump(model, open("model.pkl", "wb"))

print("Model and vectorizer saved successfully!")

Model and vectorizer saved successfully!


In [11]:
# 10. Manual Test
# ================================
sample = ["this is an excellent book and I love reading it"]
sample_vec = tfidf.transform(sample)
print("Sample Prediction:", model.predict(sample_vec))

Sample Prediction: [1]
